In [2]:
!pip3 install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [26]:
import cv2
import face_recognition
import numpy as np

In [33]:
rachel_image = face_recognition.load_image_file("rachel.png")
rachel_encoding = face_recognition.face_encodings(rachel_image)[0]

xander_image = face_recognition.load_image_file("xander.jpg")
xander_encoding = face_recognition.face_encodings(xander_image)[0]

# james_image = face_recognition.load_image_file("james.jpg")
# james_encoding = face_recognition.face_encodings(james_image)[0]

In [34]:
known_face_encodings = [
    rachel_encoding,
    xander_encoding
]
known_face_names = [
    "Rachel Chen"
    "Xander Chin"
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True


In [35]:
video_capture = cv2.VideoCapture(1)
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = np.ascontiguousarray(frame[:, :, ::-1])

    # Find all the faces in the frame of video
    face_locations = face_recognition.face_locations(rgb_frame)

    # Proceed only if faces are detected
    if face_locations:
        # Encode the faces found in the frame
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

            name = "Unknown"

            # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()